In [3]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import time

# Function to scrape data from Wikipedia page
def scrape_wikipedia(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    text = soup.get_text().lower()

    # Count word frequencies
    word_frequency = {}
    for word in text.split():
        if word.isalpha():
            word_frequency[word] = word_frequency.get(word, 0) + 1

    return word_frequency

# Function to store data in the SQLite database
def store_data_in_database(word_frequency):
    conn = sqlite3.connect('word_frequency.db')
    cursor = conn.cursor()

    # Create table if it doesn't exist
    cursor.execute('''CREATE TABLE IF NOT EXISTS word_frequency
                      (word TEXT PRIMARY KEY, frequency INTEGER)''')

    # Insert data into the table
    for word, frequency in word_frequency.items():
        cursor.execute('INSERT OR REPLACE INTO word_frequency VALUES (?, ?)', (word, frequency))

    conn.commit()
    conn.close()

# Function to fetch the highest frequency of top 30 words
def fetch_highest_frequency():
    conn = sqlite3.connect('word_frequency.db')
    cursor = conn.cursor()

    # Fetch the highest frequency of top 30 words
    cursor.execute('SELECT word, frequency FROM word_frequency ORDER BY frequency DESC LIMIT 30')
    highest_frequency = cursor.fetchall()

    conn.close()

    return highest_frequency

# Function to calculate the average frequency of all words
def calculate_average_frequency():
    conn = sqlite3.connect('word_frequency.db')
    cursor = conn.cursor()

    # Fetch all word frequencies
    cursor.execute('SELECT frequency FROM word_frequency')
    frequencies = cursor.fetchall()

    # Calculate average frequency
    average_frequency = sum(frequency for frequency, in frequencies) / len(frequencies)

    conn.close()

    return ('Average', average_frequency)

if __name__ == "__main__":
    # Scrape data from Wikipedia pages
    url1 = "https://en.wikipedia.org/wiki/Special:Random"
    url2 = "https://en.wikipedia.org/wiki/Special:Random"
    data1 = scrape_wikipedia(url1)
    data2 = scrape_wikipedia(url2)

    # Merge data from both pages and store in the database
    merged_word_frequency = {**data1, **data2}
    store_data_in_database(merged_word_frequency)

    # Fetch the highest frequency and the average frequency
    highest_frequency = fetch_highest_frequency()
    average_frequency = calculate_average_frequency()

    # Combine both data into a list
    data_to_write = highest_frequency + [average_frequency]

    # Authenticate with Google Sheets API using credentials
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
    client = gspread.authorize(creds)

    # Open the Google Sheet by its ID
    sheet_id = "Your Spreadsheet id"  # Replace with the actual ID of your Google Sheet
    sheet = client.open_by_key(sheet_id).sheet1

    # Write the data to the Google Sheet
    sheet.insert_rows(data_to_write, row=2)
